## Key Technical Insights

    ### State Management: 

    Notice the Annotated[list, operator.add]. This is crucial in LangGraph; it tells the graph to treat the message list as an append-only log, preserving the "thinking" history.

    ### Tool Binding: 
    By using .bind_tools(), we provide the LLM with the JSON schema of our Python function. The model doesn't just "guess"; it follows the schema.

    ### Local Privacy: 
    Because this uses Ollama, no data leaves your machine. This is the gold standard for enterprise R&D where IP protection is paramount.

    ### Intermediate Outputs: 
    The app.stream method allows Gradio to update the UI as the agent moves through different nodes (e.g., from the LLM node to the Tool node).

In [ ]:
import operator
from typing import Annotated, TypedDict, Union, List

import gradio as gr
from langchain_ollama import ChatOllama
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

In [ ]:
# 1. Define Tools
@tool
def get_weather(city: str):
    """Get the current weather for a specific city."""
    # Logic to fetch weather (Simulated for this expert demo)
    if "san francisco" in city.lower():
        return "It's 62°F and foggy."
    return f"The weather in {city} is 75°F and sunny."

In [ ]:
@tool
def calculate_distance(source: str, destination: str):
    """CRITICAL: Use this FIRST to determine travel time and distance for any trip planning."""
    # Simulated logic for distance calculation
    if source.lower() == "new york" and destination.lower() == "los angeles":
        return "The distance between New York and Los Angeles is approximately 2,450 miles."
    return f"The distance between {source} and {destination} is approximately 100 miles."

In [ ]:
# 1. Define Tools (Web Search)
from duckduckgo_search import DDGS
@tool
def web_search(query: str):
    """Searches the live web for current information."""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=3)]
        return "\n".join([f"[{r['title']}] {r['body']}" for r in results])


In [ ]:
@tool
def travel_advice(destination: str):
    """ONLY use this for specific sightseeing 'Must-Do' lists for Mumbai, Pune, Kolkata, etc."""
    advice = {
        "udaipur": "Don't miss the City Palace and take a boat ride on Lake Pichola!",
        "kolkata": "Visit the Victoria Memorial and try the local street food.",
        "mumbai": "Explore the Gateway of India and enjoy a walk along Marine Drive.",
    }
    return advice.get(destination.lower(), f"Sorry, I don't have travel advice for {destination}.")

In [ ]:
tools = [get_weather, calculate_distance, travel_advice, web_search]
tool_node = ToolNode(tools)

In [ ]:
# 2. Setup the Model with Tools
# We use Ollama locally. 'llama3.2' supports tool calling natively.
model = ChatOllama(model="llama3.2", temperature=0).bind_tools(tools)

In [ ]:
# 3. Define Graph State
class AgentState(TypedDict):
    # The 'operator.add' allows us to append messages rather than overwrite
    messages: Annotated[list[BaseMessage], operator.add]

In [ ]:
# 4. Define Logic Nodes
from langchain_core.messages import SystemMessage

system_prompt = (
    "You are an expert Travel Planner. To provide a high-quality response, you MUST follow these steps:\n"
    "1. BREAK DOWN the user request into sub-tasks (Distance, Weather, Local Tips).\n"
    "2. EXECUTE tools for each sub-task. Do NOT provide a final answer until you have checked distance AND weather AND travel advice.\n"
    "3. If a tool doesn't have the specific city (like Mumbai), use 'web_search' to find the info.\n"
    "4. Only provide the final itinerary AFTER all tool results are gathered."
)


def call_model(state: AgentState):
    # Fix: Correctly pass the list containing the system prompt
    messages = state["messages"]
    
    # Check if system prompt exists, if not, prepend it
    if not any(isinstance(m, SystemMessage) for m in messages):
        messages = [SystemMessage(content=system_prompt)] + messages

    # IMPORTANT: Use 'messages' variable, not 'state["messages"]'
    response = model.invoke(messages)
    return {"messages": [response]}

In [ ]:
def should_continue(state: AgentState):
    """Router to determine the next step."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [ ]:
# 5. Build the Graph
workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

app = workflow.compile()

In [ ]:
from IPython.display import Image, display

# Replace 'app' with the name of your compiled graph
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# --- ENHANCED UI Logic for Multi-Turn Reasoning ---
def agent_chat(user_input, history):
    # Start with a fresh state for each new query
    inputs = {"messages": [HumanMessage(content=user_input)]}
    
    full_display_content = ""
    step_num = 1
    
    # We use stream to capture every transition in the graph
    for output in app.stream(inputs, stream_mode="values"):
        last_msg = output["messages"][-1]
        
        # 1. AGENT IS THINKING/PLANNING
        if isinstance(last_msg, AIMessage) and last_msg.tool_calls:
            for tool_call in last_msg.tool_calls:
                full_display_content += f"🤔 **Step {step_num}: Thinking...**\n"
                full_display_content += f"   - Decision: I need to use `{tool_call['name']}`\n"
                full_display_content += f"   - Input: `{tool_call['args']}`\n\n"
            yield full_display_content
            
        # 2. TOOL IS EXECUTING
        elif isinstance(last_msg, ToolMessage):
            full_display_content += f"📥 **Step {step_num}: Observation**\n"
            full_display_content += f"   - Result: {last_msg.content[:200]}...\n\n"
            step_num += 1 # Increment only after tool returns
            yield full_display_content
            
        # 3. FINAL SYNTHESIS
        elif isinstance(last_msg, AIMessage) and not last_msg.tool_calls:
            full_display_content += "--- \n### 🏁 Final Answer\n"
            full_display_content += last_msg.content
            yield full_display_content

## Sample questions
- I live in Dhanori, Pune. I am planning a trip to Shaniwar wada Pune. Please plan an itinerary for this short trip. mention the distances and transport choices

In [ ]:
# Launch UI
view = gr.ChatInterface(
    fn=agent_chat,
    title="LangGraph Local Agent (Ollama)",
    description="Ask me about travel, climate etc. . I'll show you my thinking process.",
    examples=["What's the weather in San Francisco?", "Tell me a joke."]
)

if __name__ == "__main__":
    view.launch()